In [ ]:
import weaviate

# Connect to the local instance deployed with Docker Compose
client = weaviate.connect_to_local()

client.is_ready()

In [ ]:
import weaviate, os

# Connect to a Weaviate Cloud instance
client = weaviate.connect_to_wcs(
    cluster_url=os.getenv("WORKSHOP_DEMO_URL"),
    auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WORKSHOP_DEMO_KEY_ADMIN")),
)

client.is_ready()

## Create a collection with no vectorizer

In [ ]:
# Note: in practice, you shouldn't rerun this cell, as it deletes your data
# in "MyCollection", and then you need to re-import it again.
import weaviate.classes.config as wc

# Delete the collection if it already exists
if (client.collections.exists("MyCollection")):
    client.collections.delete("MyCollection")

client.collections.create(
    name="MyCollection",

    vectorizer_config=wc.Configure.Vectorizer.none(), # No vectorizer needed

    vector_index_config=wc.Configure.VectorIndex.hnsw( # Optional
        distance_metric=wc.VectorDistances.COSINE # select prefered distance metric
    ),

    properties=[ # defining properties (data schema) is optional
        wc.Property(name="title", data_type=wc.DataType.TEXT), 
        wc.Property(name="points", data_type=wc.DataType.INT),
    ]
)

print("Successfully created collection: MyCollection.")

## Insert an object with a vector

In [ ]:
my_collection = client.collections.get("MyCollection")
my_collection.data.insert(
    properties={
        "title": "First Object",
        "points": 11, 
    },
    vector=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
)

In [ ]:
res = my_collection.query.fetch_objects(include_vector=True)

print(res.objects[0].properties)
print(res.objects[0].vector)

## Insert many objects with their vectors using batch

In [ ]:
source = [
    {
        "title": "Second Object",
        "points": 22,
        "vector": [0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
    },
    {
        "title": "Third Object",
        "points": 33,
        "vector": [0.3, 0.1, -0.1, -0.3, -0.5, -0.7]
    },
    {
        "title": "Fourth Object",
        "points": 44,
        "vector": [0.4, 0.41, 0.42, 0.43, 0.44, 0.45]
    },
    {
        "title": "Fifth Object",
        "points": 55,
        "vector": [0.5, 0.5, 0, 0, 0, 0]
    },
]

In [ ]:

with my_collection.batch.dynamic() as batch:
    for item in source:
        batch.add_object(
            properties={
                "title": item["title"],
                "points": item["points"],
            },
            vector=item["vector"]
        )

## Example with insert_many

In [ ]:
# sample_data = [
#    wc.DataObject(
#       properties={
#          "title": "First Object",
#          "points": 11, 
#       },
#       vector=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
#    ),
#    wc.DataObject(
#       properties={
#          "title": "Second Object",
#          "points": 22,
#       },
#       vector=[0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
#    ),
#    wc.DataObject(
#       properties={
#          "title": "Third Object",
#          "points": 33,
#       },
#       vector=[0.3, 0.1, -0.1, -0.3, -0.5, -0.7]
#    ),
#    wc.DataObject(
#       properties={
#          "title": "Fourth Object",
#         "points": 44,
#       },
#       vector=[0.4, 0.41, 0.42, 0.43, 0.44, 0.45]
#    ),
#    wc.DataObject(
#       properties={
#          "title": "Fifth Object",
#          "points": 55,
#       },
#       vector=[0.5, 0.5, 0, 0, 0, 0]
#    ),
# ]

# my_collection.data.insert_many(sample_data)

## Query
Available types of queries you can run when working with vector embeddings (without modules) in **Weaviate**:

1. [near_vector](https://weaviate.io/developers/weaviate/search/similarity#search-with-a-vector)

2. [near_object](https://weaviate.io/developers/weaviate/search/similarity#search-with-an-existing-object)

### nearVector Example
**First example** - Search Weaviate with a vector embedding, and return title property.

See [the docs](https://weaviate.io/developers/weaviate/search/similarity#search-with-a-vector) for more.

In [ ]:
response = my_collection.query.near_vector(
    near_vector=[-0.012, 0.021, -0.23, -0.42, 0.5, 0.5],
    limit=2,
)

for item in response.objects:
    print(item.uuid)
    print(item.properties, "\n")

**Second example** - The same search query, but this time also return `distance`, and `vector`.

In [ ]:
from weaviate.classes.query import MetadataQuery

response = my_collection.query.near_vector(
    near_vector=[-0.012, 0.021, -0.23, -0.42, 0.5, 0.5],
    include_vector=True,
    return_metadata=MetadataQuery(distance=True),
    limit=2,
)

for item in response.objects:
    print(item.properties)
    print(item.metadata.distance)
    print(item.vector, "\n")

**Third example** – Same vector query, but this time we will filter on "points" (which should be greater than 44). Also, let's return "title" and "points".

See [the docs](https://weaviate.io/developers/weaviate/search/filters#filter-with-one-condition) for more.

In [ ]:
import weaviate.classes.query as wq

response = my_collection.query.near_vector(
    near_vector=[-0.012, 0.021, -0.23, -0.42, 0.5, 0.5],
    return_metadata=wq.MetadataQuery(distance=True),
    filters=wq.Filter.by_property("points").greater_than(30),
    limit=2,
)

for item in response.objects:
    print(item.properties)
    print(item.metadata.distance, "\n")

### nearObject Example

Weaviate also allows you to search for similar objects.

See [the docs](https://weaviate.io/developers/weaviate/search/similarity#search-with-an-existing-object) for more.

**Fourth example** - 
Search through `MyCollection` for similar objects, by providing an id from the previous query. 

> Note #1: The id was taken from the query above <br/>
> The generated id for you might be different.

> Note #2: The first object returned is always itself.

In [ ]:
import weaviate.classes.query as wq

response = my_collection.query.near_object(
    near_object="34b3c69c-355f-430e-9dd5-366bb8de44e0",
    return_metadata=wq.MetadataQuery(distance=True),
    limit=3,
)

for item in response.objects:
    print(item.uuid)
    print(item.properties)
    print(item.metadata.distance, "\n")

## Close the client when done

In [ ]:
client.close()